In [ ]:
from config import *
import pandas as pd
import numpy as np
q_std_map = pd.read_csv('F:/Projects/data/corpus/qa/FinanceFAQ/input/q_std.tsv', sep='\t', encoding='utf-8')['0'].to_dict()
query_pair = pd.read_csv('F:/Projects/data/corpus/qa/FinanceFAQ/input/query_pair_0.tsv', sep='\t', encoding='utf-8')
query_pair['q_std'] = query_pair['q_std'].map(q_std_map)
query_pair.to_csv(fst_train_file, sep='\t', encoding='utf-8', index=False)
query_pair.iloc[5:9]

In [ ]:
query_pair.q_sim.str.len().describe()

In [ ]:
# 为每个q_std随机选择一个q_sim作为dev集
q_std_list = query_pair['q_std'].unique().tolist()
query_pair['test_rnd'] = query_pair.q_std.apply(lambda x: np.random.rand())
query_pair['nrank_test'] = query_pair.groupby('q_std')['test_rnd'].rank(ascending=0, method='first')
dev_query_pair =  query_pair[query_pair.nrank_test<=1][['q_std', 'q_sim']]
dev_query_pair.head(5)

In [ ]:
# 为所有的query配一个qid
q_std_dev = dev_query_pair.q_std.unique().tolist()
q_sim_dev = dev_query_pair.q_sim.unique().tolist()
q_qid = q_std_dev + q_sim_dev
q_qid = list(set(q_qid))
q_qid_dict = {i+1:q_qid[i] for i in range(0, len(q_qid))}  # {id: query}
q_qid_dict_inv = {v: k for k, v in q_qid_dict.items()}  # {query: id}

# 建立ir_corpus： [q_id, q_std]的映射
ir_corpus = {q_qid_dict_inv[v]: v for v in q_std_list if v not in q_sim_dev}
ir_corpus_df = pd.DataFrame(list(ir_corpus.items()), columns=['qid', 'question']).sort_values('qid').reset_index(drop=True)
ir_corpus_df.to_csv(ir_path, sep='\t', index=False)

# 保存dev
dev_query_pair['qid'] = dev_query_pair.q_sim.map(q_qid_dict_inv)
dev_query_pair['duplicate_qids'] = dev_query_pair.q_std.map(q_qid_dict_inv)
dev_query_pair.duplicate_qids = dev_query_pair.duplicate_qids.astype('str')
dev_query_pair = dev_query_pair.groupby(['q_sim', 'qid']).apply(lambda v: ','.join(v['duplicate_qids'])).reset_index(name='duplicate_qids')[['qid', 'q_sim', 'duplicate_qids']]
dev_query_pair.to_csv(fst_dev_file, sep='\t', index=False)

In [ ]:
print('读取数据集并分别保存标问、相似问、所有语料: ', fst_train_file)
std_data = pd.read_csv(fst_train_file, sep="\t")
q_std_list = std_data.q_std.unique().tolist()  # 标准问list
q_sim_list = std_data.q_sim.unique().tolist()  # 相似问list
q_corpus = list(set(q_std_list + q_sim_list))

q_std_df = pd.DataFrame(q_std_list, columns=['q'])
q_corpus_df = pd.DataFrame(q_corpus, columns=['q'])
q_sim_df = pd.DataFrame(q_sim_list, columns=['q'])

q_std_df.to_csv(q_std_file, index=None, header=False, sep="\t")
q_corpus_df.to_csv(q_corpus_file, index=None, header=False, sep="\t")
q_sim_df.to_csv(q_sim_file, index=None, header=False, sep="\t")

print('q_std_list:——>', len(q_std_list), 'q_sim_list:——>', len(q_sim_list), 'q_corpus:——>', len(q_corpus))